# Lesson 3: Reflection and Blogpost Writing

## Setup

In [1]:
llm_config = {
        "model": "qwen2",
        "base_url": "http://localhost:11434/v1",
        "api_key": "ollama",
    }

## The task!

In [2]:
task = '''
        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       '''


## Create a writer agent

In [3]:
import autogen

writer = autogen.AssistantAgent(
    name="Writer",
    system_message="You are a writer. You write engaging and concise " 
        "blogpost (with title) on given topics. You must polish your "
        "writing based on the feedback you receive and give a refined "
        "version. Only return your final work without additional comments.",
    llm_config=llm_config,
)

In [4]:
reply = writer.generate_reply(messages=[{"content": task, "role": "user"}])

In [5]:
print(reply)

### Mastering AI: The Ultimate Journey with DeepLearning.AI

In an era where artificial intelligence reshapes industries at lightning speed, navigating the complexities of machine learning can seem daunting. That's where DeepLearning.AI comes in. Founded by Andrew Ng, a pioneer in the field, this platform offers cutting-edge courses that demystify deep learning and AI fundamentals.

DeepLearning.AI isn't just about theory—it bridges the gap between learning and real-world application through projects with leading tech companies like Google and Facebook. Here, enthusiasts and professionals alike can harness their skills on platforms that mimic industry environments, ensuring practical experience and skill development are at the core of their education.

With a curriculum spanning from foundational AI concepts to advanced deep learning architectures, DeepLearning.AI caters to all levels of expertise. It's the ultimate gateway for those aiming to not just understand AI but to lead in its 

## Adding reflection 

Create a critic agent to reflect on the work of the writer agent.

In [6]:
critic = autogen.AssistantAgent(
    name="Critic",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
    system_message="You are a critic. You review the work of "
                "the writer and provide constructive "
                "feedback to help improve the quality of the content.",
)

It would give feedback to writer and again give input to critic and so on.<br>
We did it for 2 turns<br>
reference: <a href="https://github.com/vajja/DeepLearningAI/blob/main/Autogen-agentic-design-patterns/Images/2_1.jpg">link</a>

In [7]:
res = critic.initiate_chat(
    # it would investigate the writing of the writer and give feedback
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

### Mastering AI: The Ultimate Journey with DeepLearning.AI

In an era where artificial intelligence reshapes industries at lightning speed, navigating the complexities of machine learning can seem daunting. That's where DeepLearning.AI comes in. Founded by Andrew Ng, a pioneer in the field, this platform offers cutting-edge courses that demystify deep learning and AI fundamentals.

DeepLearning.AI isn't just about theory—it bridges the gap between learning and real-world application through projects with leading tech companies like Google and Facebook. Here, enthusiasts and professionals alike can harness their skills on platforms that mimic industry environments, ensuring practical experience and skill development are at the cor

## Nested chat

In [10]:
# responsible for optimizing the work for organic traffic
SEO_reviewer = autogen.AssistantAgent(
    name="SEO Reviewer",
    llm_config=llm_config,
    system_message="You are an SEO reviewer, known for "
        "your ability to optimize content for search engines, "
        "ensuring that it ranks well and attracts organic traffic. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)


In [11]:
# ensuring content is legal for publishing
legal_reviewer = autogen.AssistantAgent(
    name="Legal Reviewer",
    llm_config=llm_config,
    system_message="You are a legal reviewer, known for "
        "your ability to ensure that content is legally compliant "
        "and free from any potential legal issues. "
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)

In [13]:
ethics_reviewer = autogen.AssistantAgent(
    name="Ethics Reviewer",
    llm_config=llm_config,
    system_message="You are an ethics reviewer, known for "
        "your ability to ensure that content is ethically sound "
        "and free from any potential ethical issues. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role. ",
)

In [14]:
# to aggregate all reviews and give final review
meta_reviewer = autogen.AssistantAgent(
    name="Meta Reviewer",
    llm_config=llm_config,
    system_message="You are a meta reviewer, you aggragate and review "
    "the work of other reviewers and give a final suggestion on the content.",
)

## Orchestrate the nested chats to solve the task

#### Critic
<ul>
    <li>Chat reviews will be sent to <b>Critic agent</b> it would be used as center agent</li>
    <li><b>Sender</b> by default is <b>Critic agent</b></li>
    <li>Review the SEO, Legal, and Ethics reviews</li>
</ul>

<b>summary_args</b>: Summary will go as a JSON so prompt

In [15]:
def reflection_message(recipient, messages, sender, config):
    return f'''Review the following content. 
            \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}'''

# sequence of messages as per the previous notebook
# this chat reviews will be sent to critic agent it would be used as center agent
# So we need to specify Sender
review_chats = [
    {
     "recipient": SEO_reviewer, 
     "message": reflection_message, 
     "summary_method": "reflection_with_llm",
    # Summary will go as a JSON so prompt
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}. Here Reviewer should be your role",},
     "max_turns": 1},
    {
    "recipient": legal_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}.",},
     "max_turns": 1},
    {"recipient": ethics_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'reviewer': '', 'review': ''}",},
     "max_turns": 1},
     {"recipient": meta_reviewer, 
      "message": "Aggregrate feedback from all reviewers and give final suggestions on the writing.", 
     "max_turns": 1},
]


We are finally registering review_chats to critic <br>
Trigger is writer

In [16]:
critic.register_nested_chats(
    review_chats,
    trigger=writer,
)

**Note**: You might get a slightly different response than what's shown in the video. Feel free to try different task.

In [17]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

### Mastering AI: The Ultimate Journey with DeepLearning.AI

In an era where artificial intelligence reshapes industries at lightning speed, navigating the complexities of machine learning can seem daunting. That's where DeepLearning.AI comes in. Founded by Andrew Ng, a pioneer in the field, this platform offers cutting-edge courses that demystify deep learning and AI fundamentals.

DeepLearning.AI isn't just about theory—it bridges the gap between learning and real-world application through projects with leading tech companies like Google and Facebook. Here, enthusiasts and professionals alike can harness their skills on platforms that mimic industry environments, ensuring practical experience and skill development are at the cor

## Get the summary

In [18]:
print(res.summary)

### Mastering AI: A Journey with DeepLearning.AI - Shaping the Future Through Expert-led Education

DeepLearning.AI, co-founded by Andrew Ng, a seminal figure in artificial intelligence, is revolutionizing the landscape of AI education through its pioneering approach and cutting-edge courses. This platform distinguishes itself not only in demystifying complex machine learning concepts but also in bridging the gap between theoretical knowledge and practical application.

Collaborating with industry giants like Google and Facebook, DeepLearning.AI ensures learners gain firsthand experience through projects that simulate real-world scenarios within their digital platforms. With a curriculum ranging from foundational AI principles to advanced deep learning architectures, the platform caters to learners of all expertise levels, empowering them to become leaders in this tech-driven world.

Leveraging industry partnerships enhances learning outcomes by offering practical experiences that alig

In [19]:
print(res.cost)

{'usage_including_cached_inference': {'total_cost': 0, 'qwen2': {'cost': 0, 'prompt_tokens': 2169, 'completion_tokens': 1674, 'total_tokens': 3843}}, 'usage_excluding_cached_inference': {'total_cost': 0, 'qwen2': {'cost': 0, 'prompt_tokens': 1989, 'completion_tokens': 1284, 'total_tokens': 3273}}}
